# **LOFAR and NuRadio**

This script highlights how to use NuRadioReco for reading in LOFAR Data, how to save them as a .nur file and read them in again. It also shows how to access traces and other information stored in the event object. For more LOFAR examples and information on how to contribute, please refer to [the internal LOFAR documentation](https://gitlab.science.ru.nl/radio-cr/lofar-cr/-/blob/main/Software%20development/HowToContribute.md).

The first part shows how to generate a NRR file from LOFAR data. You will need to generate a NRR event before being able to execute the second part of the notebook, which shows one simple example of accessing single traces, and one more advanced example in which electric fields, event parameters and antenna positions are accessed and coordinate transformations with `radiotools` are being performed.

## Imports:


In [ ]:
import socket
import radiotools

import NuRadioReco.detector.detector
import NuRadioReco.modules.io.eventWriter
import NuRadioReco.modules.io.eventReader
import NuRadioReco.modules.eventTypeIdentifier

from NuRadioReco.modules import channelBandPassFilter
from NuRadioReco.modules import voltageToEfieldConverter
from NuRadioReco.utilities import units
from NuRadioReco.framework.parameters import stationParameters, channelParameters, showerParameters

from NuRadioReco.modules.io.LOFAR import readLOFARData
from NuRadioReco.modules.LOFAR import (
    stationRFIFilter,
    stationGalacticCalibrator,
    stationPulseFinder,
    planeWaveDirectionFitter_LOFAR,
    pipelineVisualizer_LOFAR
)

from astropy.time import Time
from pathlib import Path

# Creating a `.nur` file from LOFAR data

## LOFAR data and information you need

LOFAR data is stored in TBB files. The LOFAR reader in `NuRadioReco.modules.io.LOFAR.readLOFARData` provides an interface to convert this data into a NRR event.
We will have a look at the event with `event_id` 92380604. If you don't have access to LOFAR data, the following cell will download some sample data to the current folder.

In [ ]:
from NuRadioReco.utilities.dataservers import download_from_dataserver

data_path = './data/lofar' # where to store the data - this will drop the data in `./data/lofar`
target_path = f'{data_path}/sample-data.tar.gz'
if not Path(data_path, 'tbb').exists(): # do not download again if the data has already been downloaded
    download_from_dataserver(remote_path='lofar_share/lofar-sample-event.tar.gz', target_path=target_path)

You can save the data in different stages during the pipeline, in case you want to look at raw data. The `save_stage` is a set which defines all the steps at which a `.nur` file should written out to disk. If you only want to save a `.nur` file at the very end, use an empty set `save_stage = {}`.  
Similarly, the `station_set` can be used to select which stations to read in. If you want to read in data for all stations, set `station_set = None`.
This script per default will save the `.nur` file in your current folder, if you want to save it anywhere else, set `nur_save_path` to your desired path.
The `event_id` identifies an event based on its timestamp.

In [ ]:
station_set = ["CS002", "CS003", "CS004", "CS005", "CS006", "CS007"] # stations to read in
save_stage = {} #{"raw", "filtered", "calibrated"}  # stages to save
nur_save_path = str(Path.cwd()) + '/'
event_id = 92380604

## The event Writer

The eventWriter is the object we use to write an `Event` object into a `.nur` file. Here we initialise it, and later we will call the `begin()` and `run()` functions to actually use it. 

In [ ]:
eventWriter = NuRadioReco.modules.io.eventWriter.eventWriter()

## Initialising the reader

If you are on coma, where the LOFAR data is stored, the reader will automatically access the directories with the necessary files. Otherwise you need to specify where the TBB, JSON and metadata directories with the necessary files are. In this example, the fallback location is `/mnt/coma`. This could be for example the location where you mount the directories onto your local filesystem (using a tool like `sshfs`). More information is in the internal LOFAR documentation.

In [ ]:
import logging
logger = logging.getLogger('NuRadioReco')
# logger.setLevel(logging.DEBUG) # uncomment to print ALL debug messages. The default is logging.STATUS

In [ ]:
machine_name = socket.getfqdn()
if "coma" in machine_name or "science.ru.nl" in machine_name:
    reader = readLOFARData.readLOFARData(
        restricted_station_set=station_set
    )

else: # this assumes you have downloaded the data to `data_path`
    reader = readLOFARData.readLOFARData(
        restricted_station_set=station_set,
        tbb_directory=f"{data_path}/tbb",
        json_directory=f"{data_path}/lora/json",
        metadata_directory=f"{data_path}/metadata",
    )

reader.begin(event_id)

### Reading the detector description

Here we load the detector description of LOFAR. It is part of the NRR repository (located at `NuRadioReco/detector/LOFAR`), so we can call it with a simple path. The `LOFAR.json` contains the positions of the LOFAR stations and antennas and other important information regarding the antenna model etc. The detector is passed to the `run()` function of the LOFAR reader.

In [ ]:
# Load the LOFAR detector description
detector = NuRadioReco.detector.detector.Detector(
    "LOFAR/LOFAR.json", source="json", antenna_by_depth=False
)

##  Running the reader

We only run a single event but the run() function yields a generator, so we use a loop construction to get the event and can run modules on the event in the loop.

Structure:

- **Event type identifyer**: since we know that LOFAR events are cosmic rays, we force set the event type via the eventTypeIdentifyer. This is necessary to set certain shower parameters internally. Sets the type of event. If you save the event after this, you will see raw, uncalibrated and unfiltered data.
- **RFI filtering**: filters out narrowband RFI noise, using the phase stability method.
- **Bandpass filter**: First applies a Gaussian bandpass filter to the data in the frequency domain, then also tapers off the edges in time using a half-Hann window. The `self.begin()` is called in init function, and does not have to be explicitly called. If you save the event after this, you will get uncalibrated, filtered traces.
- **Galactic calibrator**: converts ADC counts to voltages based on the Galactic calibration. If you save after this, you will get calibrated traces.
- **Station pulse finder**: Finds stations and channels with a good signal, flags bad channels.
- **Planewave fit**: Performs a timing fit based on a simple plane wave assumption per station and saves the direction reconstruction parameters in the stationParameters.
- **Voltage to Efield converter**: converts the voltages to E-Fields using the antenna response and reconstructed directions. Saves the Electric fields separately in the event. The converter runs per channel group (even/odd antenna) since two polarizations are needed to reconstruct the E-Field.

Lastly, the full event is saved in a `.nur` file. You are now done reading the data in and can access it via the `eventReader`


In [ ]:
# initialize all the classes that we use
eventTypeIdentifier = NuRadioReco.modules.eventTypeIdentifier.eventTypeIdentifier()
rfi_filter = stationRFIFilter.stationRFIFilter()
bandpass_filter = channelBandPassFilter.channelBandPassFilter()
calibrator = stationGalacticCalibrator.stationGalacticCalibrator()
pulse_finder = stationPulseFinder.stationPulseFinder()
fitter = planeWaveDirectionFitter_LOFAR.planeWaveDirectionFitter()
converter = voltageToEfieldConverter.voltageToEfieldConverter()

In [ ]:
for event in reader.run(detector):

    eventTypeIdentifier.begin()
    for station in event.get_stations():
        eventTypeIdentifier.run(
            event, station, mode="forced", forced_event_type="cosmic_ray"
        )

    # Save the event if required
    if "raw" in save_stage:
        eventWriter.begin(nur_save_path + f"{event_id}_raw.nur")
        eventWriter.run(event, detector)
        eventWriter.end()

    # Apply RFI cleaning and bandpass filters
    rfi_filter.begin(reader=reader, rfi_cleaning_trace_length=8192)
    rfi_filter.run(event)
    rfi_filter.end()

    for station in event.get_stations():
        bandpass_filter.run(
            event,
            station,
            detector,
            passband=[30 * units.MHz, 80 * units.MHz],
            filter_type="gaussian_tapered",
            roll_width=2.5 * units.MHz,
        )
        bandpass_filter.run(
            event,
            station,
            detector,
            passband=[30 * units.MHz, 80 * units.MHz],
            filter_type="hann_tapered",
            half_hann_percent=0.1,
        )

    # Save the event if required
    if "filtered" in save_stage:
        eventWriter.begin(nur_save_path + f"{event_id}_filtered.nur")
        eventWriter.run(event, detector)
        eventWriter.end()

    # Calibrate the traces based on Galactic noise
    calibrator.begin()
    calibrator.run(event, detector)
    calibrator.end()

    # Save the event if required
    if "calibrated" in save_stage:
        eventWriter.begin(nur_save_path + f"{event_id}_calibrated.nur")
        eventWriter.run(event, detector)
        eventWriter.end()

    # Find stations with significant pulse and corresponding good channels in those stations
    pulse_finder.begin()
    pulse_finder.run(event, detector)
    pulse_finder.end()

    # Update the direction by
    fitter.begin(
        debug=False
    )
    fitter.run(event, detector)
    fitter.end()

    # Unfold the voltages to electric fields
    converter.begin()
    for station in event.get_stations():
        if station.get_parameter(stationParameters.triggered):
            for group_id in station.get_channel_ids(return_group_ids=True):
                converter.run(
                    event, station, detector,
                    use_channels=[channel.get_id() for channel in station.iter_channel_group(group_id)]
                )
    converter.end()

    eventWriter.begin(nur_save_path + f"{event_id}.nur")
    eventWriter.run(event, detector)
    eventWriter.end()

# Reading a `.nur` File

if you just want to look at the event, you can use the `pipelineVisualiser` module which automatically gives you debug plots. A nice feature of the `.nur` files is that we have option to save the `Detector` object inside of it. However, when we retrieve this the detector time is not longer set. So we first have to add that back, using the `LOFAR_event_id_to_unix` function which does the conversion from event ID to UNIX time for us.

In [ ]:
reader = NuRadioReco.modules.io.eventReader.eventReader()
reader.begin(nur_save_path + f"{event_id}.nur", read_detector=True)

detector = reader.get_detector()
detector.update(Time(readLOFARData.LOFAR_event_id_to_unix(event_id), format="unix"))

for event in reader.run():
    visualize = pipelineVisualizer_LOFAR.pipelineVisualizer()
    visualize.begin()
    visualize.run(event, detector, polarization=True, direction=True)
    visualize.end()

## Accessing specific event information

If you want to access specific traces, electric fields or parameters stored in your event, here are two examples of how to do it.

### 1 (basic) getting single traces:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

reader = NuRadioReco.modules.io.eventReader.eventReader()
reader.begin(nur_save_path + f"{event_id}.nur", read_detector=True)

detector = reader.get_detector()
detector.update(Time(readLOFARData.LOFAR_event_id_to_unix(event_id), format="unix"))

for event in reader.run():
    station = event.get_station(2) # station 2 is the station_id of CS002

    channel_even = station.get_channel(2000000) # 2000000 is the even channel id

    trace_even = channel_even.get_trace()
    times_even = channel_even.get_times()

    channel_odd = station.get_channel(2000001)

    trace_odd = channel_odd.get_trace()
    times_odd = channel_odd.get_times()

    pulse_window_start, pulse_window_end = channel_odd.get_parameter(channelParameters.signal_regions)
    fig, ax = plt.subplots(figsize=(7,5))

    ax.plot(times_even, trace_even, label='even')
    ax.plot(times_odd, trace_odd, label='odd')
    ax.set_xlabel('Time [ns]')
    ax.set_ylabel('Voltage [V]')
    ax.set_xlim([times_even[pulse_window_start], times_even[pulse_window_end]])
    ax.legend()
    plt.tight_layout()
    plt.show()

    freqs_even = channel_even.get_frequencies() / units.MHz # get frequencies in MHz
    spec_even = channel_even.get_frequency_spectrum()

    freqs_odd = channel_odd.get_frequencies() /  units.MHz # get frequencies in MHz
    spec_odd = channel_odd.get_frequency_spectrum()

    fig, ax = plt.subplots(figsize=(7,5))

    ax.plot(freqs_even, np.abs(spec_even), label='even')
    ax.plot(freqs_odd, np.abs(spec_odd), label='odd')
    ax.set_xlabel('Frequency [MHz]')
    ax.set_ylabel('Amplitude [V/Hz]')
    ax.set_xlim([30, 80])
    ax.legend()
    plt.tight_layout()
    plt.show()

### 2 (more advanced) plotting polarization footprint of the event:
This script is taken from the `pipelineVisualizer` but it is a nice and condensed example that shows how to access electric fields, shower parameters such as the zenith and azimuth, and how to use the coordinate transformations from `radiotools`.
It accesses the electric fields in the stations, calculates the polarization angle and degree of polarization via `get_stokes`, converts the positions of the electric fields into shower coordinates and then plots the polarization of each antenna along with the core.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import Normalize
import numpy as np

reader = NuRadioReco.modules.io.eventReader.eventReader()
reader.begin(nur_save_path + f"{event_id}.nur", read_detector=True)

detector = reader.get_detector()
detector.update(Time(readLOFARData.LOFAR_event_id_to_unix(event_id), format="unix"))

for event in reader.run():
        
    from NuRadioReco.framework.electric_field import get_stokes

    fig_pol, ax = plt.subplots(figsize=(8,7))

    triggered_station_ids = [
        station.get_id() for station in event.get_stations() if station.get_parameter(stationParameters.triggered)
    ]
    num_stations = len(triggered_station_ids)

    cmap = colormaps['jet']  
    norm = Normalize(vmin=0, vmax=num_stations-1) 

    lora_core = event.get_hybrid_information().get_hybrid_shower("LORA").get_parameter(showerParameters.core)

    try:
        core = event.get_first_shower().get_parameter(showerParameters.core)
    except KeyError:
        core = lora_core
        
    for i, station in enumerate(event.get_stations()):
        
        if station.get_parameter(stationParameters.triggered):

            zenith = station.get_parameter(stationParameters.cr_zenith) / units.rad
            azimuth = station.get_parameter(stationParameters.cr_azimuth) / units.rad

            cs = radiotools.coordinatesystems.cstrafo(
                zenith, azimuth, magnetic_field_vector=None, site="lofar"
            )

            efields = station.get_electric_fields()

            station_pos = detector.get_absolute_position(station.get_id())
            station_pos_vB = cs.transform_to_vxB_vxvxB(station_pos, core=core)[0]
            station_pos_vvB = cs.transform_to_vxB_vxvxB(station_pos, core=core)[1]

            ax.scatter(
                station_pos_vB, station_pos_vvB,
                color=cmap(norm(i)), s=20, label=f'Station CS{station.get_id():03d}'
            )

            for field in efields:

                ids = field.get_channel_ids()
                pos = station_pos + detector.get_relative_position(station.get_id(), ids[0])

                pos_vB = cs.transform_to_vxB_vxvxB(pos, core=core)[0]
                pos_vvB = cs.transform_to_vxB_vxvxB(pos, core=core)[1]

                pulse_window_start, pulse_window_end = station.get_channel(ids[0]).get_parameter(channelParameters.signal_regions)
                pulse_window_len = pulse_window_end - pulse_window_start

                trace = field.get_trace()[:,pulse_window_start:pulse_window_end]

                efield_trace_vxB_vxvxB = cs.transform_to_vxB_vxvxB(
                    cs.transform_from_onsky_to_ground(trace)
                )

                #get stokes parameters
                stokes = get_stokes(*efield_trace_vxB_vxvxB[:2], window_samples=64)

                stokes_max = np.argmax(stokes[0])

                I = stokes[0,stokes_max]
                Q = stokes[1,stokes_max]
                U = stokes[2,stokes_max]
                V = stokes[3,stokes_max]

                # get stokes uncertainties by picking a pure noise value
                I_sigma = stokes[0, stokes_max-pulse_window_len//4]
                Q_sigma = stokes[1, stokes_max-pulse_window_len//4]
                U_sigma = stokes[2, stokes_max-pulse_window_len//4]
                V_sigma = stokes[3, stokes_max-pulse_window_len//4]

                pol_angle = 0.5 * np.arctan2(U,Q)
                pol_angle_sigma= np.sqrt((U_sigma**2*(0.5*Q/(U**2+Q**2))**2 + Q_sigma**2*(0.5*U/(U**2+Q**2))**2))

                # if the polarization deviates from the vxB direction by more than 80 degrees,
                # this could indicate something wrong with the antenna. Show a warning including
                # the channel ids
                if np.abs(0.5 * np.arctan2(U,Q)) > 80*np.pi/180:
                    print("strange polarization direction in channel group %s" % ids)

                pol_degree= np.sqrt(U**2 + Q**2 + V**2) / I
                pol_degree *= 7 # scale for better visibility

                dx = pol_degree * np.cos(pol_angle)
                dy = pol_degree * np.sin(pol_angle)

                dx_sigma_plus = pol_degree * np.cos(pol_angle + pol_angle_sigma)
                dy_sigma_plus = pol_degree * np.sin(pol_angle + pol_angle_sigma)

                dx_sigma_minus = pol_degree * np.cos(pol_angle - pol_angle_sigma)
                dy_sigma_minus = pol_degree * np.sin(pol_angle - pol_angle_sigma)

                ax.arrow(
                    pos_vB, pos_vvB, dx_sigma_plus, dy_sigma_plus,
                    head_width=2, head_length=5,
                    fc=cmap(norm(i)), ec = cmap(norm(i)), alpha=0.5
                )
                ax.arrow(
                    pos_vB, pos_vvB, dx_sigma_minus, dy_sigma_minus,
                    head_width=2, head_length=5,
                    fc=cmap(norm(i)), ec = cmap(norm(i)), alpha=0.5
                )
                ax.arrow(
                    pos_vB, pos_vvB, dx, dy,
                    head_width=2, head_length=6,
                    fc=cmap(norm(i)), ec = cmap(norm(i))
                )

    if np.any(core != lora_core):
        lora_vB = cs.transform_to_vxB_vxvxB(lora_core, core=core)[0]
        lora_vvB = cs.transform_to_vxB_vxvxB(lora_core, core=core)[1]
        ax.scatter(lora_vB, lora_vvB, color='tab:red', s=50, label='LORA core', marker = 'x')
        label = 'radio core'
    else:
        label = 'LORA core'

    ax.scatter([0], [0], color='black', s=50, label=label, marker = 'x')
    ax.legend()
    ax.set_xlabel('Direction along $v \\times B$ [m]')
    ax.set_ylabel('Direction along $v \\times (v \\times B)$ [m]')   

    plt.show(fig_pol)